In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
from yfinance import EquityQuery


#try pandas datareader?
#use yf.pdr_override() if opt in for datareader

In [ ]:
# dat = yf.Ticker("2YY=F")
# dat.info

In [ ]:
# df = yf.download(['GOOG'], period='1mo')
# #use only close price?
# #252 is the usual number of trading days
# df.head()

In [ ]:
# #Here we first create a filter with EquityQuery and then use it in the yf.screen() function
# q = EquityQuery('and', [EquityQuery('eq', ['region', 'us']),
#               #EquityQuery('gte', ['intradaymarketcap', 2000000000]),
#               EquityQuery('gt', ['dayvolume', 5000000])
# ])


# response = yf.screen(q, sortField = 'dayvolume', sortAsc = False, size=100)